# Автоматизация отчётности 

### Задачи
##### 1. Создаём своего телеграм-бота с помощью @BotFather
##### 2. Пишем скрипт для сборки отчета по ленте новостей. Отчет должен состоять из двух частей:
*  текст с информацией о значениях ключевых метрик за предыдущий день
*  график с значениями метрик за предыдущие 7 дней

##### 3. Отображаем в отчете следующие ключевые метрики: 

- DAU 
- Просмотры
- Лайки
- CTR

In [ ]:
import telegram
import pandahouse
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Создаём бота и передаём ему наш токен (из BotFather)
def telegram_report(chat=None):
    #chat_id = chat or 726885196 #Чат ID личного чат бота
    chat_id = chat or -715060805 #Чат ID группы курса (updates = bot.getUpdates() - этой функцией узнаём чат ID)
    bot = telegram.Bot(token='5388356743:AAFRvvBIu_oe-DprC7UJNiTajKpFf72QSnE') #Создаём бота и передаём ему наш токен (из BotFather)
    
    #Формируем базу данных за прошлый день и формируем нужные нам столбцы

    connection = {
        'host': 'https://clickhouse.lab.karpov.courses',
                          'database':'simulator_20220520',
                          'user':'student', 
                          'password':'dpo_python_2020'}

    q = '''SELECT
             toDate(time) as data, 
             countIf(action='view') as view, 
             countIf(action = 'like') as like, 
             count(distinct user_id) as users,
             round(like / view,2) as ctr
           FROM simulator_20220520.feed_actions 
           WHERE data BETWEEN today() - 7 AND today()
           GROUP BY data '''

    df = pandahouse.read_clickhouse(q, connection=connection)
    
    #Формируем сообщение для бота, в котором будут прописаны все метрики
    msg = (f' Current data report {df.data[6]}: ') + '\n'\
         + (f'DAU:  {df.users[6]}') + '\n'\
         + (f'View: {df.view[6]}') + '\n'\
         + (f'Like: {df.like[6]}') + '\n'\
         + (f'CTR: {df.ctr[6]}') 

    bot.sendMessage(chat_id, text = msg)   
    
    #Строим графики по нашим метрикам

    plt.figure(figsize = (15,17))
    sns.set_style("whitegrid")

    plt.subplot(3,1,1)
    sns.lineplot(data = df, x = 'data', y = 'users')
    plt.title('DAU last week')

    plt.subplot(3,1,2)
    sns.lineplot(data = df, x = 'data', y = 'view')
    plt.subplot(3,1,2)
    sns.lineplot(data = df, x = 'data', y = 'like', label = 'a')
    plt.ylabel('View and Like')
    plt.title('View and Like last week')
    plt.legend(labels = ['View', 'Like'])

    plt.subplot(3,1,3)
    sns.lineplot(data = df, x = 'data', y = 'ctr')
    plt.title('CTR last week') 

    #Сохраняем наш график как файловый объет в буфере, чтобы потом была возможность доставать объет из буфера
    import io #позволяет направлять потоки ввода вывода данных
    plot_object = io.BytesIO()
    plt.savefig(plot_object)
    plot_object.name = 'metric_plot.png'
    plot_object.seek(0)

    plt.close()

    bot.sendPhoto(chat_id = chat_id, photo = plot_object)
    
try:
    telegram_report()
except Exception:
    print(Exception)